In [1]:
import os
import re
from pathlib import Path
from PyPDF2 import PdfReader
from market_agents.memory.config import load_config_from_yaml
from market_agents.memory.setup_db import DatabaseConnection
from market_agents.memory.embedding import MemoryEmbedder
from market_agents.memory.knowledge_base import MarketKnowledgeBase
from market_agents.memory.knowledge_base_agent import KnowledgeBaseAgent
from market_agents.memory.vector_search import MemoryRetriever
from market_agents.agents.market_agent import MarketAgent
from market_agents.inference.message_models import LLMConfig
from market_agents.environments.mechanisms.chat import ChatEnvironment


In [2]:
# Load config
config_path = Path("../market_agents/memory/memory_config.yaml")
memory_config = load_config_from_yaml(str(config_path))

# Initialize database connection and embedding service
db_conn = DatabaseConnection(memory_config)
embedder = MemoryEmbedder(memory_config)

# Initialize knowledge base with "hamlet" prefix
hamlet_kb = MarketKnowledgeBase(
    config=memory_config,
    db_conn=db_conn,
    embedding_service=embedder,
    table_prefix="hamlet_kb"
)

In [ ]:
# Read PDF content
pdf_path = "/Users/interstellarninja/Documents/projects/MarketAgents/hamlet_PDF_FolgerShakespeare.pdf"
pdf_reader = PdfReader(pdf_path)
text_content = ""
for page in pdf_reader.pages:
    page_text = page.extract_text()
    cleaned_text = re.sub(r'FTLN \d+', '', page_text)
    cleaned_text = re.sub(r'^\d+\s*', '', cleaned_text, flags=re.MULTILINE)
    text_content += cleaned_text

# Store in knowledge base with metadata
knowledge_id = hamlet_kb.ingest_knowledge(
    text_content,
    metadata={
        "source": "Hamlet - Folger Shakespeare Library",
        "type": "play",
        "author": "William Shakespeare"
    }
)
print(f"Successfully created Hamlet knowledge base with ID: {knowledge_id}")

In [3]:
# Initialize retriever
retriever = MemoryRetriever(memory_config, db_conn, embedder)

# Create knowledge base agent
kb_agent = KnowledgeBaseAgent(
    market_kb=hamlet_kb,
    retriever=retriever
)

In [ ]:

# Test some queries
def query_hamlet(question: str):
    print(f"\nQuestion: {question}")
    print("-" * 80)
    
    results = kb_agent.retrieve(question, "hamlet_kb")
    
    for i, chunk in enumerate(results, 1):
        print(f"\nResult {i} (Similarity: {chunk.similarity:.3f})")
        print("Text:", chunk.text)
        print("\nContext:", chunk.context)
        print("-" * 80)

# Try different queries
queries = [
    "What is Hamlet's famous 'To be or not to be' soliloquy about?",
    "How does Ophelia die in the play?",
    "What is the role of the ghost in the play?",
    "Describe the relationship between Hamlet and Horatio"
]

for query in queries:
    query_hamlet(query)

In [4]:
import json


chat_env = ChatEnvironment(name="hamlet_chat")

hamlet_agent = MarketAgent.create(
    memory_config=memory_config,
    db_conn=db_conn,
    agent_id="hamlet_agent",
    use_llm=True,
    llm_config=LLMConfig(
        client="openai",
        model="gpt-4o",
        temperature=0.5,
        max_tokens=1024
    ),
    knowledge_agent=kb_agent, 
    environments={"chat": chat_env},
)

# Add a test message to the chat
hamlet_agent.task = "What is Hamlet's famous 'To be or not to be' soliloquy about?"
chat_env.mechanism.add_user_message(hamlet_agent.task)

# Generate a response
perception = await hamlet_agent.perceive(environment_name="chat")
print(json.dumps(perception, indent=4))

action = await hamlet_agent.generate_action(environment_name="chat", perception=perception)
print(json.dumps(action, indent=4))

reflection = await hamlet_agent.reflect(environment_name="chat")
print(json.dumps(reflection, indent=4))




Cognitive Memory Results:
Memory 1:
{'cognitive_step': 'action', 'content': '{"agent_id": "agent_123", "action": {"thoughts": [{"reasoning": "The current perception indicates a pattern of polite greetings and inquiries about well-being, which suggests that the focus should be on maintaining a friendly interaction. The unrelated Shakespearean text should not distract from this goal."}, {"reasoning": "Given the strategy to engage users politely and reciprocate their inquiries, the action should involve crafting a response that acknowledges the user\'s greeting and asks about their well-being."}, {"reasoning": "Since the confidence level is reasonably high at 0.85, I should proceed with a friendly and engaging response while ignoring the unrelated textual data."}], "final_answer": "Hello! How are you doing today? I hope you\'re having a wonderful day. \\ud83d\\ude0a"}}'}

Memory 2:
{'cognitive_step': 'reflection', 'content': '"In this scenario, the action taken was to maintain a friendly